# Project RFM - Melbourne

RFM is abbreviation for Recency, Frequency and Monetary. 

It is a technique that helps determine marketing and sales strategies based on customers’ buying habits.

- Recency: Time passed since the customer’s last purchase. In other words, it is the “time passed since the last contact of the customer”.

<center>Recency= RFM analysis date — Last purchase date</center>

- Frequency: Total number of purchases. It shows how frequently the customer does shopping. It can be found from the number of the invoices that one customer has.

- Monetary (Monetary Value): Total spending by the customer.


Customer segmentation is the process of separating these values into groups by scoring between 1 and 5. Depending on these scores, the customers are segmented into different groups. These groups can be shown on the Recency and Frequency Grid as the following:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt

# to display all columns and rows:
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);

#to arrange the decimals
pd.set_option('display.float_format', lambda x: '%.2f' % x) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Proyectos/Australia/sample.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
df['date'] =  pd.to_datetime(df['date'])
df.head()

,invoice,date,customer_id,stockcode,price
0,1,2020-07-25,d18734,y7,50.45
1,2,2020-01-17,c21086,x7,25.30
2,3,2019-07-05,d18185,z5,18.40
3,4,2019-02-26,c18331,z2,5.50
4,5,2019-02-10,b16309,y7,18.40


In [ ]:
# read dataset
# df = pd.read_csv('sample.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
# df['date'] =  pd.to_datetime(df['date'])
# df.head()

In [ ]:
df.shape

(100000, 5)

In [ ]:
# print a concise summary of a DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      100000 non-null  int64         
 1   date         100000 non-null  datetime64[ns]
 2   customer_id  100000 non-null  object        
 3   stockcode    100000 non-null  object        
 4   price        100000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 3.8+ MB


In [ ]:
# count number of distinct ids, ignore NaN value
df["invoice"].nunique()

100000

In [ ]:
# How many prices are there?
df["price"].value_counts().head()

50.45    20180
25.30    20146
12.99    20001
18.40    19849
5.50     19824
Name: price, dtype: int64

In [ ]:
#sorting the products from the most purhased product to the least along with their prices: 
df.groupby("date").agg({"price":"sum"}).sort_values("date", ascending = False).head(10)

,price
date,
2020-12-31,2975.00
2020-12-30,3370.69
2020-12-29,3768.99
2020-12-28,2914.32
2020-12-27,2692.86
2020-12-26,3719.75
2020-12-25,2898.33
2020-12-24,3746.14
2020-12-23,3682.10


In [ ]:
df.groupby(["date","invoice"] ).agg({"price":"sum"}).sort_values("price", ascending=True).head()

,,price
date,invoice,
2020-10-07,51869,5.50
2020-03-10,15065,5.50
2019-04-18,74656,5.50
2020-03-10,17426,5.50
2019-04-18,74243,5.50


In [ ]:
df.isnull().sum()

invoice        0
date           0
customer_id    0
stockcode      0
price          0
dtype: int64

In [ ]:
 #This is another way to do the same thing as above
df.sort_values("date", ascending=True).head() 

,invoice,date,customer_id,stockcode,price
2316,2317,2019-01-01,c19511,y4,50.45
88380,88381,2019-01-01,a16609,z5,18.40
41675,41676,2019-01-01,e20346,z7,5.50
41603,41604,2019-01-01,a16487,x4,12.99
41493,41494,2019-01-01,d19076,z5,50.45


In [ ]:
#how much money we gained?
df.groupby("date").agg({"price":"sum"}).sort_values("price", ascending = False).head()

,price
date,
2019-09-23,4173.63
2020-06-16,3998.65
2020-09-12,3935.83
2019-10-12,3916.61
2019-02-24,3914.46


# Data Preparation 

In [ ]:
df.describe([0.01,0.05,0.10,0.25,0.50,0.75,0.90,0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
invoice,100000.00,50000.50,28867.66,1.00,1000.99,5000.95,10000.90,25000.75,50000.50,75000.25,90000.10,95000.05,99000.01,100000.00
price,100000.00,22.62,15.43,5.50,5.50,5.50,5.50,12.99,18.40,25.30,50.45,50.45,50.45,50.45


In [ ]:
#we got the outlier values for our information, but we will not use it because we do not build a model
for feature in ['price']:

    Q1 = df[feature].quantile(0.01)
    Q3 = df[feature].quantile(0.99)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
    else:
        print(feature, "no")

price no


### Recency

In [ ]:
df["date"].min(), df["date"].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2020-12-31 00:00:00'))

In [ ]:
#assigning today date as the 1 day after the max date
today_date = dt.datetime(2021,1,1)

In [ ]:
df.groupby('customer_id').agg({"date":"max"}).head()

,date
customer_id,
a15600,2020-08-01
a15601,2020-07-15
a15602,2020-02-19
a15603,2020-12-10
a15604,2020-12-25


In [ ]:
len(set(df['customer_id'])) == df.shape[0]

False

In [ ]:
(today_date - df.groupby('customer_id').agg({'date':'max'})).head()

,date
customer_id,
a15600,153 days
a15601,170 days
a15602,317 days
a15603,22 days
a15604,7 days


In [ ]:
temp_df = (today_date - df.groupby('customer_id').agg({'date':'max'}))

In [ ]:
#changing the name of date to Recency 
temp_df.rename(columns={"date": "Recency"}, inplace = True)  
recency_df = temp_df["Recency"].apply(lambda x: x.days)
recency_df.head()

customer_id
a15600    153
a15601    170
a15602    317
a15603     22
a15604      7
Name: Recency, dtype: int64

### Frequency

In [ ]:
freq_df = df.groupby(['customer_id']).agg({'invoice':'count'})
freq_df.rename(columns={"invoice": "Frequency"}, inplace = True)
freq_df.head()

,Frequency
customer_id,
a15600,3
a15601,2
a15602,3
a15603,3
a15604,5


### Money

In [ ]:
monetary_df = df.groupby('customer_id').agg({'price':'sum'})
monetary_df.rename(columns={"price": "Monetary"}, inplace = True)
monetary_df.head()

,Monetary
customer_id,
a15600,101.05
a15601,11.00
a15602,106.40
a15603,43.79
a15604,93.49


In [ ]:
# Series, df, df
print(recency_df.shape, freq_df.shape, monetary_df.shape)

(33004,) (33004, 1) (33004, 1)


In [ ]:
#Lets bring all the values together under a new data frame called 'rfm':
rfm = pd.concat([recency_df, freq_df, monetary_df],  axis=1)
rfm.head()

,Recency,Frequency,Monetary
customer_id,,,
a15600,153,3,101.05
a15601,170,2,11.00
a15602,317,3,106.40
a15603,22,3,43.79
a15604,7,5,93.49


In [ ]:
#Normally the smallest of the recency scoring, which is 1, is the best recency score. 
#However, we will define this in reverse and put the value 5 as the best recency value so that it will be the same as the others, 
#so score 5 will be the most recent and the best recency score:

rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels = [5, 4, 3, 2, 1])
rfm["RecencyScore"].value_counts()

3    6649
5    6632
1    6595
4    6588
2    6540
Name: RecencyScore, dtype: int64

In [ ]:
rfm["Frequency"].describe([0.01,0.05,0.10,0.25,0.50,0.60,0.75,0.90,0.95, 0.99]).T

count   33004.00
mean        3.03
std         1.57
min         1.00
1%          1.00
5%          1.00
10%         1.00
25%         2.00
50%         3.00
60%         3.00
75%         4.00
90%         5.00
95%         6.00
99%         7.00
max        11.00
Name: Frequency, dtype: float64

In [ ]:
cut_bins = [0,1,2,3,5,11]

rfm["FrequencyScore"] = pd.cut(rfm["Frequency"], bins = cut_bins, labels = [1, 2, 3, 4, 5])
rfm["FrequencyScore"].value_counts()

4    8814
2    8362
3    7831
1    5595
5    2402
Name: FrequencyScore, dtype: int64

In [ ]:
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels = [1, 2, 3, 4, 5])
rfm["MonetaryScore"].value_counts()

2    7052
1    6688
5    6559
4    6540
3    6165
Name: MonetaryScore, dtype: int64

In [ ]:
# RFM SCORE
rfm["RFM_SCORE"] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str) + rfm['MonetaryScore'].astype(str)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
customer_id,,,,,,,
a15600,153,3,101.05,3,3,4,334
a15601,170,2,11.00,3,2,1,321
a15602,317,3,106.40,2,3,5,235
a15603,22,3,43.79,5,3,2,532
a15604,7,5,93.49,5,4,4,544


In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,33004.00,211.23,175.34,1.00,69.00,164.00,313.25,731.00
Frequency,33004.00,3.03,1.57,1.00,2.00,3.00,4.00,11.00
Monetary,33004.00,68.53,44.37,5.50,31.48,62.19,94.24,315.69


In [ ]:
#the champions:
rfm[rfm["RFM_SCORE"] == "555"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
customer_id,,,,,,,
a15614,3,6,118.14,5,5,5,555
a15631,36,7,244.10,5,5,5,555
a15664,52,6,130.45,5,5,5,555
a15781,42,7,181.49,5,5,5,555
a15814,43,7,137.13,5,5,5,555


In [ ]:
rfm[rfm["RFM_SCORE"] == "555"].shape

(811, 7)

In [ ]:
#the worst customers:
rfm[rfm["RFM_SCORE"] == "111"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
customer_id,,,,,,,
a15612,513,1,12.99,1,1,1,111
a15615,705,1,18.40,1,1,1,111
a15622,400,1,12.99,1,1,1,111
a15645,524,1,25.30,1,1,1,111
a15684,650,1,25.30,1,1,1,111


In [ ]:
rfm[rfm["RFM_SCORE"] == "111"].shape

(2242, 7)

In [ ]:
#To segment the customers using “Recency” and “Frequency” values: 
#set up regular expression (regex) structure by using dictionaries to name Customer Segments according to 
#Recency and Frequency Scores:

seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Lose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [ ]:
#recency ve frequency points are obtained with the following code 
#(as the monetary is a similar value to frequency, it is not considered here):
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm['Segment'].head()

customer_id
a15600    33
a15601    32
a15602    23
a15603    53
a15604    54
Name: Segment, dtype: object

In [ ]:
#Pull the value corresponding to the ranges of the score in the dictionary defined in 'seg_map', 
#and add it as a new column named "Segment":

rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
customer_id,,,,,,,,
a15600,153,3,101.05,3,3,4,334,Need Attention
a15601,170,2,11.00,3,2,1,321,About to Sleep
a15602,317,3,106.40,2,3,5,235,At Risk
a15603,22,3,43.79,5,3,2,532,Potential Loyalists
a15604,7,5,93.49,5,4,4,544,Champions


In [ ]:
#Retrieve statistical values of recency, frequency and monetary values by segment classes:
rfm[["Segment", "Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"])

Recency       Frequency       Monetary      
                       mean count      mean count     mean count
Segment                                                         
About to Sleep       167.91  2382      1.69  2382    38.45  2382
At Risk              332.20  4736      3.54  4736    80.77  4736
Can't Lose           285.49   271      6.38   271   143.30   271
Champions             25.84  3389      5.02  3389   112.60  3389
Hibernating          428.11  8128      1.52  8128    34.00  8128
Loyal Customers      120.55  5563      4.83  5563   109.26  5563
Need Attention       163.94  1764      3.00  1764    67.40  1764
New Customers         28.35   435      1.00   435    22.72   435
Potential Loyalists   57.76  5848      2.57  5848    58.70  5848
Promising             89.32   488      1.00   488    22.79   488

In [ ]:
rfm[rfm["Segment"] == "Need Attention"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
customer_id,,,,,,,,
a15600,153,3,101.05,3,3,4,334,Need Attention
a15621,202,3,101.05,3,3,4,334,Need Attention
a15643,203,3,151.35,3,3,5,335,Need Attention
a15651,136,3,63.59,3,3,3,333,Need Attention
a15652,176,3,31.48,3,3,2,332,Need Attention


In [ ]:
rfm[rfm["Segment"] == "Need Attention"].shape[0]

1764

In [ ]:
need_attention_df = pd.DataFrame()
need_attention_df["Need_Attention_CustomerID"] = rfm[rfm["Segment"] == "Need Attention"].index
need_attention_df.head()

,Need_Attention_CustomerID
0,a15600
1,a15621
2,a15643
3,a15651
4,a15652


In [ ]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
customer_id,,,,,,,,
a15600,153,3,101.05,3,3,4,334,Need Attention
a15601,170,2,11.00,3,2,1,321,About to Sleep
a15602,317,3,106.40,2,3,5,235,At Risk
a15603,22,3,43.79,5,3,2,532,Potential Loyalists
a15604,7,5,93.49,5,4,4,544,Champions


In [ ]:
rw2 = rfm.copy()
rw2['Recency'] = rw2['Recency']/rw2['Recency'].max()
rw2['Frequency'] = rw2['Frequency']/rw2['Frequency'].max()
rw2['Monetary'] = rw2['Monetary']/rw2['Monetary'].max()
rw2.drop(['RecencyScore','FrequencyScore','MonetaryScore','RFM_SCORE'], axis=1, inplace=True)
rw2 = rw2.reset_index()
rw2.head()

,customer_id,Recency,Frequency,Monetary,Segment
0,a15600,0.21,0.27,0.32,Need Attention
1,a15601,0.23,0.18,0.03,About to Sleep
2,a15602,0.43,0.27,0.34,At Risk
3,a15603,0.03,0.27,0.14,Potential Loyalists
4,a15604,0.01,0.45,0.30,Champions


In [ ]:
rw2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33004 entries, 0 to 33003
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  33004 non-null  object 
 1   Recency      33004 non-null  float64
 2   Frequency    33004 non-null  float64
 3   Monetary     33004 non-null  float64
 4   Segment      33004 non-null  object 
dtypes: float64(3), object(2)
memory usage: 1.3+ MB


In [ ]:
rw2['Segment'].value_counts()

Hibernating            8128
Potential Loyalists    5848
Loyal Customers        5563
At Risk                4736
Champions              3389
About to Sleep         2382
Need Attention         1764
Promising               488
New Customers           435
Can't Lose              271
Name: Segment, dtype: int64